In [24]:
import pandas as pd
import numpy as np
import re
import string
import time
from datetime import datetime
from datetime import date
import os

#Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import metrics

# Save model
import pickle
import joblib

In [25]:
# fileLocation = 'C:\\Users\\VictorY\\Desktop\\TestData\\stage3FinalProcessedDF_Nov-17-2019.csv'
# moviesDataFileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Final_Data_Movies_Directors_old.csv'
# moviesDataFileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Final_data_movies_director.csv'
# moviesDataFileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Movies_Directors_WithAwards.csv'

#Dynamic path location
# moviesDataFileLocation = os.path.join(os.path.realpath(os.path.pardir),'datasets\\data1_Data_Movies.csv')
# moviesDataFileLocation = os.path.join(os.path.realpath(os.path.pardir),'datasets\\data2_Movies_WithAwards.csv')

fileName = '1_Director_Data_Mar-21-2020'
DataFileLocation = os.path.join(os.path.realpath(os.path.pardir),'datasets\\'+ fileName +'.csv')


saveFileToPath = "C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\"

movies_data = pd.read_csv(DataFileLocation)

In [26]:
# movies_data = movies_data.sample(n=10000, random_state = 42)

In [27]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41629 entries, 0 to 41628
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   director_ids              41629 non-null  object 
 1   director_name             41629 non-null  object 
 2   movies_produced           41629 non-null  int64  
 3   average_rating_value      41629 non-null  float64
 4   average_rating_count      41629 non-null  float64
 5   career_Length             41629 non-null  int64  
 6   Gender                    41629 non-null  object 
 7   birthYear                 41629 non-null  int64  
 8   primaryProfession         41629 non-null  object 
 9   knownForTitles            41629 non-null  object 
 10  CareerStartYear           41629 non-null  int64  
 11  median_inter_event_time   41629 non-null  float64
 12  average_movie_per_year    41629 non-null  float64
 13  average_inter_event_time  41629 non-null  float64
dtypes: flo

In [28]:
movies_data.Gender.value_counts()

Male      32638
Female     8991
Name: Gender, dtype: int64

In [29]:
numericalGender = {"Male": 1, "Female": 0 }
movies_data.Gender.replace(numericalGender,inplace=True)

In [30]:
object_features_to_remove = movies_data.select_dtypes(include = 'object').columns.values.tolist()
# object_features_to_remove.remove('Gender')
object_features_to_remove

['director_ids', 'director_name', 'primaryProfession', 'knownForTitles']

In [31]:
# req_columns = ['review_count_user', 'review_count_critic','rating_value',
#                'rating_count','movie_year','birthYear','Gender']

In [32]:
# movies_data = movies_data[req_columns]
movies_data = movies_data[movies_data.columns.difference(object_features_to_remove)]
# movies_data.drop(columns = ['Gender_Male','Gender_Female'],inplace = True)

print('The shape of our Unbalanced Movies dataset is:', movies_data.shape)

The shape of our Unbalanced Movies dataset is: (41629, 10)


In [33]:
# movies_data = movies_data[req_columns]
# print('The shape of our Director specific dataset is:', movies_data.shape)

# Feature Selection

1. Logistic Regression
2. SVM Classifier
3. Gradient Boost Classifier
4. XG Boost Classifier

In [34]:
def quartile_bin(df,class_feature='rating_value'):
    # Quartile cut into 4 different buckets

    bin_quartiles = ['terrible','poor','average','excellent']
    cutIntoQuartiles = pd.qcut(df[class_feature].to_list(), q=4,labels=bin_quartiles)
    
    print(cutIntoQuartiles.categories)
    
    df['class'] = cutIntoQuartiles
    df['class'] = df['class'].astype(object)
    print('The shape of our dataset before dropping the class feature is:', df.shape)
    df.drop(columns = [class_feature] , inplace = True)
    print('The shape of our dataset after dropping the class feature  is:', df.shape)
    return df

In [35]:
from sklearn.feature_selection import RFE

In [36]:
movies_data.columns

Index(['CareerStartYear', 'Gender', 'average_inter_event_time',
       'average_movie_per_year', 'average_rating_count',
       'average_rating_value', 'birthYear', 'career_Length',
       'median_inter_event_time', 'movies_produced'],
      dtype='object')

## 1) Feature selection through : Logistic Regression

In [37]:
df = quartile_bin(movies_data,'average_rating_count')

Index(['terrible', 'poor', 'average', 'excellent'], dtype='object')
The shape of our dataset before dropping the class feature is: (41629, 11)
The shape of our dataset after dropping the class feature  is: (41629, 10)


In [38]:
df 

,CareerStartYear,Gender,average_inter_event_time,average_movie_per_year,average_rating_value,birthYear,career_Length,median_inter_event_time,movies_produced,class
0,1946,1,0.9,1.11,7.36,1918,63,1.0,70,excellent
1,1961,1,0.0,1.00,7.20,1924,1,0.0,1,excellent
2,1967,1,0.0,1.00,5.70,1925,1,0.0,1,excellent
3,1957,1,0.0,1.00,5.80,1899,1,0.0,1,excellent
4,1973,1,2.0,0.67,5.85,1916,3,2.0,2,excellent
...,...,...,...,...,...,...,...,...,...,...
41624,2015,1,0.0,1.00,9.90,1982,1,0.0,1,terrible
41625,2016,0,0.0,1.00,10.00,1999,1,0.0,1,terrible
41626,2016,1,0.0,1.00,9.20,1990,1,0.0,1,poor
41627,2016,0,0.0,1.00,9.70,1995,1,0.0,1,terrible


In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
X = movies_data.drop("class", axis='columns')   #Feature Matrix
y = movies_data["class"]          #Target Variable

In [41]:
X.columns.values

array(['CareerStartYear', 'Gender', 'average_inter_event_time',
       'average_movie_per_year', 'average_rating_value', 'birthYear',
       'career_Length', 'median_inter_event_time', 'movies_produced'],
      dtype=object)

In [42]:
logistic_model_estimator = LogisticRegression()
selector = RFE(logistic_model_estimator, n_features_to_select = None , step = 1) 

In [43]:
selector = selector.fit(X, y)
print("Support : " , selector.support_)
print("\nRanking : " , selector.ranking_)

C:\Users\yuvar\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\yuvar\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

Support :  [False False  True False  True False  True  True False]

Ranking :  [5 6 1 2 1 4 1 1 3]


C:\Users\yuvar\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [44]:
f = selector.get_support(1) #the most important features
f

array([2, 4, 6, 7], dtype=int64)

In [45]:
imporant_features_X_df = X[X.columns[f]] # final features`
imporant_features_X_df

,average_inter_event_time,average_rating_value,career_Length,median_inter_event_time
0,0.9,7.36,63,1.0
1,0.0,7.20,1,0.0
2,0.0,5.70,1,0.0
3,0.0,5.80,1,0.0
4,2.0,5.85,3,2.0
...,...,...,...,...
41624,0.0,9.90,1,0.0
41625,0.0,10.00,1,0.0
41626,0.0,9.20,1,0.0
41627,0.0,9.70,1,0.0


In [23]:
imporant_features_X_df.columns.values

array(['CareerStartYear', 'average_rating_count', 'birthYear',
       'career_Length', 'movies_produced'], dtype=object)

## 2) Feature Selection : SVM Classifier

In [ ]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt

In [ ]:
movies_data_new = movies_data.sample(1000,random_state = 42)
movies_data_new.shape

In [ ]:
X = movies_data_new.drop("Gender", axis='columns')   #Feature Matrix
y = movies_data_new["Gender"]          #Target Variable

In [ ]:
svc_estimator = SVC(kernel="linear")
svc_rfe = RFE(estimator = svc_estimator, n_features_to_select = None , step=1)
svc_rfe = svc_rfe.fit(X, y)

In [ ]:
# RFE parameters
svc_rfe.get_params

In [ ]:
svc_rfe.score(X,y)

In [ ]:
print("Support : " , svc_rfe.support_)
print("Ranking : " , svc_rfe.ranking_)

In [ ]:
f = svc_rfe.get_support(1) #the most important features
f

In [ ]:
reduced_X = X[X.columns[f]]
reduced_X

## 3) Feature Selection - Gradient Boosting

In [ ]:
# Boosting classifiers
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# movies_data_new = movies_data.sample(100,random_state = 42)
# movies_data_new.shape

In [ ]:
X = movies_data.drop("Gender", axis='columns')   #Feature Matrix
y = movies_data["Gender"]          #Target Variable

In [ ]:
gb_estimator = GradientBoostingClassifier(n_estimators=100,
                                         learning_rate=0.1,
                                         max_leaf_nodes = 32
                                         )

gb_rfe = RFE(estimator = gb_estimator, n_features_to_select = None , step=1)
gb_rfe = gb_rfe.fit(X, y)

In [ ]:
gb_rfe.score(X,y)

In [ ]:
print("Support : " , gb_rfe.support_)
print("Ranking : " , gb_rfe.ranking_)

In [ ]:
f = gb_rfe.get_support(1) #the most important features
f

In [ ]:
reduced_X = X[X.columns[f]]
reduced_X

## 4) XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
X = movies_data.drop("Gender", axis='columns')   #Feature Matrix
y = movies_data["Gender"]          #Target Variable

In [ ]:
xgb_estimator = XGBClassifier(n_estimators=100,
                        learning_rate=0.1,
                        max_features=2,
                        max_depth=2,
                        random_state= 42
                        )

xgb_rfe = RFE(estimator = xgb_estimator, n_features_to_select = None , step=1)
xgb_rfe = xgb_rfe.fit(X, y)

In [ ]:
xgb_rfe.score(X,y)

In [ ]:
print("Support : " , xgb_rfe.support_)
print("Ranking : " , xgb_rfe.ranking_)

In [ ]:
f = xgb_rfe.get_support(1) #the most important features
f

In [ ]:
reduced_X = X[X.columns[f]]
reduced_X

In [ ]:
reduced_X.columns.values

# Filter Method

Using Pearson Correlation

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = movies_data.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["Gender"])
print(cor_target)

In [ ]:
cor_target[cor_target>0.07].sort_values()

In [ ]:
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [ ]:
len(movies_data.columns)

## Optimum number of features

In [ ]:
movies_data_new = movies_data.sample(1000,random_state = 42)
movies_data_new.shape

X = movies_data_new.drop("Gender", axis='columns')   #Feature Matrix
y = movies_data_new["Gender"]          #Target Variable

In [ ]:
#no of features
nof_list=np.arange(1,len(movies_data.columns))            
high_score=0

#Variable to store the optimum features
nof=0
score_list =[]

In [ ]:
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    # Training the model
    svclassifier = SVC(kernel='linear')
    rfe = RFE(svclassifier,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    svclassifier.fit(X_train_rfe,y_train)
    score = svclassifier.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]

In [ ]:
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

##  RFE Cross validation with XGBoost Classifier

Feature ranking with recursive feature elimination and cross-validated selection of the best number of features.

In [ ]:
from xgboost import XGBClassifier
from sklearn.feature_selection import RFECV

In [ ]:
# movies_data_new = movies_data.sample(1000,random_state = 42)
# movies_data_new.shape

X = movies_data.drop("Gender", axis='columns')   #Feature Matrix
y = movies_data["Gender"]                        #Target Variable

In [ ]:
xgb_estimator = XGBClassifier(n_estimators=100,
                        learning_rate= 0.1,
                        max_features=2,
                        max_depth=2,
                        random_state= 42
                        )

xgb_rfe = RFECV(estimator = xgb_estimator, min_features_to_select = 10 , step=1 , cv = 5 )
xgb_rfe = xgb_rfe.fit(X, y)

In [ ]:
xgb_rfe.support_

In [ ]:
xgb_rfe.ranking_

In [ ]:
f = xgb_rfe.get_support(1) #the most important features
f

In [ ]:
reduced_X = X[X.columns[f]]
reduced_X

In [ ]:
reduced_X.columns.values

In [ ]:
break

In [ ]:
def train_model_XGB_classifier(X_train, X_test, y_train, y_test,learning_rate = 1):
    
    # Classifier parameters    

    xgb_clf = XGBClassifier(n_estimators=100,
                            learning_rate=learning_rate,
                            max_features=2,
                            max_depth=2,
                            random_state= 42
                           )
    xgb_clf.fit(X_train, y_train)
    
    xgb_train_score = xgb_clf.score(X_train, y_train)
    print("Training Score : ",round(xgb_train_score,3))
    
    xgb_test_score = xgb_clf.score(X_test, y_test)
    print("Testing Score : ",round(xgb_test_score,3))
    
    predictions = xgb_clf.predict(X_test)

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, predictions))

    print("Classification Report")
    print(classification_report(y_test, predictions))
    
    return xgb_clf

In [ ]:
def divide_test_train_with_scaling_methods(df,predict='Gender',scalingMethod = "MinMaxScaler"):
    
    
    # Feature matrix and target variable
    X = df.drop(predict,axis = 'columns')
    y = df[predict]
    
    # Scaling
    if (scalingMethod == "MinMaxScaler"):
        print("The scaling method used is : ",scalingMethod)
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X)
        # X_test = scaler.transform(X_test)
        
    else:
        print("No Scaling Method used or incorrect input")
        X_train = X
    
    # Divide the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size =0.20, random_state = 42 )
    
    return X_train, X_test, y_train, y_test

In [ ]:
req_columns = ['rating_value', 'rating_count', 'movie_year', 'birthYear','Gender']

In [ ]:
#movies_data.select_dtypes(include = 'object')

In [ ]:
movies_data = movies_data[req_columns]
print('The shape of our Director specific dataset is:', movies_data.shape)

In [ ]:
X_train, X_test, y_train, y_test = divide_test_train_with_scaling_methods(movies_data,scalingMethod="None")

In [ ]:
train_model_XGB_classifier(X_train, X_test, y_train, y_test,learning_rate = 0.1)